In [1]:
import pandas as pd
# from stravalib.client import Client
import stravalib
import flask
from flask import request
from stravalib import Client
import requests
import json
import webbrowser
import pandas as pd
import datetime

from bokeh.plotting import figure, output_file, show, output_notebook, ColumnDataSource
from bokeh.palettes import Spectral11
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.layouts import gridplot

import warnings
warnings.simplefilter('ignore')

In [2]:
client = Client()
# authorize_url = client.authorization_url(client_id=24706, redirect_uri='http://localhost:8282/authorized')

# url = client.authorization_url(client_id=24706,redirect_uri='http://127.0.0.1:5000/authorization')
client_id = 24706

access_token = '481cb676c1ebd336da0c25230f837ce8ac6190ad'
port = 5000
url = 'http://localhost:%d/authorized' % port
authorize_url = client.authorization_url(client_id=client_id, redirect_uri=url)

webbrowser.open(authorize_url)

code = '3c66e15a369292748e3ab195b0162487b983c6ae'

access_token = client.exchange_code_for_token(client_id=24706, client_secret=access_token, code=code)


In [3]:
# activities_returned = []
# activity_returned = dict()

In [4]:
activities_list = [(activity.start_date,activity.name,activity.distance) for activity in client.get_activities(after = "2018-01-01T00:00:00Z",  limit=800)]


No such attribute visibility on entity <Activity id=1337718957 name='Day 1 RED Jan - Muddy Connaught water' resource_state=2>
No such attribute heartrate_opt_out on entity <Activity id=1337718957 name='Day 1 RED Jan - Muddy Connaught water' resource_state=2>
No such attribute display_hide_heartrate_option on entity <Activity id=1337718957 name='Day 1 RED Jan - Muddy Connaught water' resource_state=2>
No such attribute visibility on entity <Activity id=1337718988 name='Day 2 RED Jan - Bury Road and back' resource_state=2>
No such attribute heartrate_opt_out on entity <Activity id=1337718988 name='Day 2 RED Jan - Bury Road and back' resource_state=2>
No such attribute display_hide_heartrate_option on entity <Activity id=1337718988 name='Day 2 RED Jan - Bury Road and back' resource_state=2>
No such attribute visibility on entity <Activity id=1338842063 name='Day 3 RED Jan - Shawell basin' resource_state=2>
No such attribute heartrate_opt_out on entity <Activity id=1338842063 name='Day 3 R

In [5]:
extract_df = pd.DataFrame.from_dict(activities_list)
len(extract_df)

188

In [6]:
extract_df = extract_df.rename(columns={0: 'run_date',
                          1: 'run_title',
                          2: 'run_distance_m'})

In [7]:
extract_df['kms'] = extract_df['run_distance_m']/1000

In [8]:
extract_df.index = extract_df['run_date']

In [9]:
extract_df = extract_df[['run_title', 'run_distance_m', 'kms']]

In [10]:
daily_runs_series = extract_df.resample('D')['kms'].sum()

In [11]:
daily_runs_df = pd.DataFrame(daily_runs_series)

In [12]:
def to_float(number):
    return float(number)

In [13]:
def convert_week_number(week_number):
    return f'week {week_number}'

In [14]:
def cumulative_per_week(df, week_number):
    df = df.copy(deep=True)
    df_week = df[df['week_number']==week_number]
    
    df_week['weekly_actual_total'] = df_week['kms'].sum()
    df_week['weekly_actual_cumulative'] = df_week['kms'].cumsum()
    df_week['percentage_to_total_weekly'] = df_week['weekly_actual_cumulative']/df_week['weekly_actual_total']
    df_week['weekly_goal_total'] = df_week['daily_goal'].sum()
    df_week['weekly_goal_cumulative'] = df_week['daily_goal'].cumsum()
    df_week['percentage_to_total_weekly_goal'] = df_week['weekly_goal_cumulative']/df_week['weekly_goal_total']
    
    day_number_range = range(1, len(df_week)+1)
    if len(df_week) == 7:
        day_of_week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        df_week['day_of_week'] = day_of_week
    else:
        df_week['day_of_week'] = None
        
    df_week['day_number'] = day_number_range
    
    return df_week

In [15]:
daily_runs_df['kms'] = daily_runs_df.apply(lambda row: to_float(row['kms']), axis=1)

In [16]:
daily_runs_df['cumulative_daily_actual_kms'] = daily_runs_df['kms'].cumsum()
daily_runs_df['daily_goal'] = 20/7
daily_runs_df['cumulative_daily_goal'] = daily_runs_df['daily_goal'].cumsum()
# daily_runs_df['daily_km_difference'] = 

In [17]:
daily_runs_df['daily_kms_difference'] = daily_runs_df['kms']/daily_runs_df['daily_goal']
daily_runs_df['daily_kms_cumulative_difference'] = daily_runs_df['cumulative_daily_actual_kms']/daily_runs_df['cumulative_daily_goal']

In [18]:
daily_runs_df.to_csv('strava_kms_runs_2018.csv')

In [19]:
weekly_kms_df = daily_runs_df.resample('W-MON', closed='Right', label='Right').sum()

In [20]:
weekly_kms_df = weekly_kms_df[['kms', 'daily_goal']]

In [21]:
weekly_kms_df = weekly_kms_df.rename(columns={'daily_goal': 'weekly_goal'})

In [22]:
weekly_kms_df['cumulative_weekly_kms'] = weekly_kms_df['kms'].cumsum()
weekly_kms_df['cumulative_weekly_goal_kms'] = weekly_kms_df['weekly_goal'].cumsum()

In [23]:
weekly_kms_df['week_number'] = weekly_kms_df.index.week
daily_runs_df['week_number'] = daily_runs_df.index.week

In [24]:
weekly_kms_df['week'] = weekly_kms_df.apply(lambda row: convert_week_number(row['week_number']), axis=1)
daily_runs_df['week'] = daily_runs_df.apply(lambda row: convert_week_number(row['week_number']), axis=1)

In [25]:
weekly_kms_df = weekly_kms_df.round(2)
daily_runs_df = daily_runs_df.round(2)

In [26]:
weekly_num = list(set(list(daily_runs_df['week_number'])))

In [27]:
daily_weekly_kms_df = pd.DataFrame()

In [28]:
for week in weekly_num:
    transition_df = cumulative_per_week(daily_runs_df, week)
    daily_weekly_kms_df = daily_weekly_kms_df.append(transition_df)
    

7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
2


In [43]:
daily_weekly_kms_df.to_csv('strava_data.csv')

In [29]:
output_file('grid_plot.html')

In [30]:
# output_file('strava_chart3.html')

source = ColumnDataSource(data=weekly_kms_df)

km_fig = figure(
                  plot_height=300, plot_width=600,
                  title='cumulative actual distance vs goal actual distance',
                  x_axis_label='week', y_axis_label='Kms',
                  toolbar_location=None)


km_fig.line(x='week_number', y='cumulative_weekly_kms', 
         color='blue', line_width=1,
         legend='Cumulative weekly kms',source=source)

km_fig.vbar(x='week_number', bottom=0, top='cumulative_weekly_kms', 
         color='blue', width=0.75, 
         legend='Actual', source=source)


km_fig.line(x='week_number', y='cumulative_weekly_goal_kms', 
         color='red', line_width=1,
         legend='Cumulative goal kms',source=source)

km_fig.legend.location = 'top_left'

# show(km_fig)

In [31]:
x_axis = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [32]:
day_week_cds = ColumnDataSource(daily_weekly_kms_df)


km_week_comp_fig = figure(plot_height=300, plot_width=600,
                  title='cumulative actual distance vs goal actual distance',
                  x_axis_label='Day of the week 1.0', y_axis_label='Kms',
                  x_range=x_axis,
                  toolbar_location=None, y_range=(0, 50))

week_1_view = CDSView(source=day_week_cds,
                       filters=[GroupFilter(column_name='week', group='week 1.0')])
week_2_view = CDSView(source=day_week_cds,
                        filters=[GroupFilter(column_name='week', group='week 1.0')])

km_week_comp_fig.line(x='day_of_week', y='cumulative_daily_actual_kms', 
         color='blue', line_width=1,
         legend='Cumulative',  source=day_week_cds, view=week_1_view)

km_week_comp_fig.line(x='day_of_week', y='cumulative_daily_goal', 
         color='red', line_width=1,
         legend='Cumulative goal', source=day_week_cds, view=week_2_view)

# Move the legend to the upper left corner
km_week_comp_fig.legend.location = 'top_left'

# Show the plot
# show(km_week_comp_fig)

In [33]:
weekly_kms_df['number'] = range(1, len(weekly_kms_df)+1)

In [34]:
# output_file('strava_chart3.html')

cds_bar = ColumnDataSource(data=weekly_kms_df)

# Create a figure with a datetime type x-axis
fig = figure(title='actual versus goals',
             plot_height=300, plot_width=600,
             x_axis_label='Week Number', y_axis_label='KMs',
             x_minor_ticks=2, y_range=(0, 50),
             toolbar_location=None)

# The daily words will be represented as vertical bars (columns)
fig.vbar(x='number', bottom=0, top='kms', 
         color='blue', width=0.75, 
         legend='Actual', source=cds_bar)

# The cumulative sum will be a trend line
fig.line(x='number', y='weekly_goal', 
         color='gray', line_width=1,
         legend='Goal', source=cds_bar)

# Put the legend in the upper left corner
fig.legend.location = 'top_left'

tooltips = [
            ('Kilometers','@kms'),
            ('Week number', '@week'),
           ]

# Add the HoverTool to the figure
fig.add_tools(HoverTool(tooltips=tooltips))


### Weekly bar line to show atual vs goal kms


In [35]:
# output_file('strava_chart_grid.html')

cds_bar = ColumnDataSource(data=daily_weekly_kms_df)

week_1_view = CDSView(source=cds_bar,
                       filters=[GroupFilter(column_name='week', group='week 1.0')])

# Create a figure with a datetime type x-axis
km_week_fig = figure(title='actual versus goals',
             plot_height=300, plot_width=600,
             x_axis_label='Day of the week 1.0', y_axis_label='KMs',
             x_range=x_axis,
             x_minor_ticks=2, y_range=(0, 50),
             toolbar_location=None)

# The daily words will be represented as vertical bars (columns)
km_week_fig.vbar(x='day_of_week', bottom=0, top='weekly_actual_cumulative', 
         color='blue', width=0.75, 
         legend='Actual', source=cds_bar, view=week_1_view)

# The cumulative sum will be a trend line
km_week_fig.line(x='day_of_week', y='weekly_goal_cumulative', 
         color='gray', line_width=1,
         legend='Goal', source=cds_bar, view=week_1_view)

# Put the legend in the upper left corner
km_week_fig.legend.location = 'top_left'

tooltips = [
            ('Cumulative Kilometers','@weekly_actual_cumulative'),
            ('Week number', '@week'),
           ]
# Add the HoverTool to the figure
km_week_fig.add_tools(HoverTool(tooltips=tooltips))

# Let's check it out

# all_week_gridplot = gridplot([[fig, km_fig], [km_week_comp_fig, km_week_fig]], 
#                               toolbar_location='right')
# # show(km_week_fig)
# # show(km_fig)
# # show(fig)
# show(all_week_gridplot)

In [36]:
day_week_cds = ColumnDataSource(daily_weekly_kms_df)


weeks_comp_fig = figure(plot_height=300, plot_width=600,
                  title='Week 1 vs Week 2',
                  x_axis_label='Day of the week 1.0', y_axis_label='Kms',
                  x_range=x_axis,
                  toolbar_location=None, y_range=(0, 20))

week_1_view = CDSView(source=day_week_cds,
                       filters=[GroupFilter(column_name='week', group='week 1.0')])
week_2_view = CDSView(source=day_week_cds,
                        filters=[GroupFilter(column_name='week', group='week 2.0')])

weeks_comp_fig.line(x='day_of_week', y='kms', 
         color='blue', line_width=1,
         legend='Week 1',  source=day_week_cds, view=week_1_view)

weeks_comp_fig.line(x='day_of_week', y='kms', 
         color='red', line_width=1,
         legend='Week 2', source=day_week_cds, view=week_2_view)

# Move the legend to the upper left corner
weeks_comp_fig.legend.location = 'top_left'

tooltips = [
            (' Kilometers','@kms'),
           ]
# Add the HoverTool to the figure
weeks_comp_fig.add_tools(HoverTool(tooltips=tooltips))



In [ ]:
stacked_bar = figure

In [38]:
all_week_gridplot = gridplot([[fig, km_fig], [weeks_comp_fig, km_week_fig], [None, km_week_comp_fig]], 
                              toolbar_location='right')
# show(km_week_fig)
# show(km_fig)
# show(fig)
show(all_week_gridplot)

In [39]:
output_file('select_week.html')

In [40]:
p.vbar_stack(years, x='fruits', width=0.9, color=colors, source=data,
             legend=[value(x) for x in years])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

weeks_comp_fig = figure(plot_height=300, plot_width=600,
                  title='Week 1 vs Week 2',
                  x_axis_label='Day of the week 1.0', y_axis_label='Kms',
                  x_range=x_axis,
                  toolbar_location=None, y_range=(0, 20))